*********************************************************************************************************
# A Tour of Python 3
version 0.9 (alpha)

Authors: Phil Pfeiffer, Zack Bunch, and Feyi Oyeniyi<br>
East Tennessee State University<br>
Last updated February 2020<br>

*********************************************************************************************************

# Content<a name='Contents'></a><br> 
11. [Modules](#Modules) <br> 
&ensp; 11.1  [Changing Python's default import path](#Modules-Default-Import-Path) <br> 
&ensp; 11.2  [Module caching and reimportation](#Modules-Caching-And-Reimportation)

# 11.  Modules <a name='Modules'></a>


##  11.1  Changing Python's default import path <a name='Modules-Default-Import-Path'></a>
The Python interpreter supports two types of modules:
-  built-ins, which are compiled into Python
-  file-based modules, which are accessed via the interpreter's resident file system.

Python searches for file-based modules in the directories listed in `sys.path`.  
In response to `import foo`, Python searches the directories named in `sys.path` from left to right, 
selecting the first "foo.py" in the list, much like
-  POSIX CLI's search directories named in the `PATH` environment variable for executables
-  Java's runtime engine searches directories named in `CLASSPATH` for Java class libraries.

Updating `sys.path` changes how Python searches for modules.

These examples use the following Python library resources:
-  `os.mkdir` - creates a directory
-  `sys.path` - names directories that Python will search for modules
-  `importlib.invalidate_caches` - clear Python's import cache, to assure subsequent, 'clean' executions of example
-  `shutil.rmtree` - removes a directory tree, including all files therein


In [ ]:
# 11.1.a  Accessing a module in a subdirectory of the current directory that's not in sys.path
# example should fail, since the module's directory isn't named in sys.path

# supporting constants
CREATE_AS_NEW_FILE = 'x'    # file access mode for open()

# library resources
import os, sys, importlib, shutil

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

# precondition check: ensure that we're creating a totally new, unreferenced module in a totally new directory

sample_directory_name = 'sample_directory'
corrective_action = f'please change {sample_directory_name} to a directory name that is not in sys.path'
assert sample_directory_name not in sys.path, corrective_action

sample_module_name = 'sample_module'  
corrective_action = f'please change {sample_module_name} to a module name that is not in use'
assert sample_module_name not in sys.modules.keys(), corrective_action

sample_function_name = 'sample_function' 
corrective_action = f'please change {sample_function_name} to a function name that is not in use'
assert sample_function_name not in dir(), corrective_action

importlib.invalidate_caches()   # clear stale module data, possibly from previous exercise

# preconditions established: create a module that's not on sys.path, then try to import and use it

file_to_import = sample_directory_name + '/' + sample_module_name + '.py'

try:
  os.mkdir( sample_directory_name )
  try:
    file_to_import_fd = open( file_to_import, CREATE_AS_NEW_FILE )
    try:
      return_message = f'{sample_module_name}.{sample_function_name} in {sample_directory_name}: x+y is {{x+y}}' 
      function_body  = "return f'" + return_message + "'" 
      function       = f'def {sample_function_name}(x, y):' + '\n  ' + function_body + '\n' 
      file_to_import_fd.write( function )
      file_to_import_fd.close()
      try:
        result_from_sample_function = eval( f'{sample_module_name}.{sample_function_name}(3, 4)' )
        print( result_from_sample_function )
      except Exception as exception:
        print( f"can't access {sample_module_name}: {make_printable(exception)}" )
    except Exception as exception:
      print( f"can't write to file ({file_to_import}): {make_printable(exception)}" )
    finally:
      file_to_import_fd.close()
  except Exception as exception:
    print( f"can't create file ({file_to_import}): {make_printable(exception)}" )
  finally:
    try:
      shutil.rmtree( sample_directory_name )
    except Exception as exception:
      print( f"can't remove directory ({sample_directory_name}): {make_printable(exception)}" )
except Exception as exception:
  print( f"can't create directory ({sample_directory_name}): {make_printable(exception)}" )

In [ ]:
# 11.1.b  Accessing a module in a subdirectory of the current directory that's not in sys.path
# previous example, repeated with appropriate import statement

# supporting constants
CREATE_AS_NEW_FILE = 'x'    # file access mode for open()

# library resources
import os, sys, importlib, shutil

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

# precondition check: ensure that we're creating a totally new, unreferenced module in a totally new directory

sample_directory_name = 'sample_directory'
corrective_action = f'please change {sample_directory_name} to a directory name that is not in sys.path'
assert sample_directory_name not in sys.path, corrective_action

sample_module_name = 'sample_module'  
corrective_action = f'please change {sample_module_name} to a module name that is not in use'
assert sample_module_name not in sys.modules.keys(), corrective_action

sample_function_name = 'sample_function' 
corrective_action = f'please change {sample_function_name} to a function name that is not in use'
assert sample_function_name not in dir(), corrective_action

importlib.invalidate_caches()   # clear stale module data, possibly from previous exercise

# preconditions established: create a module that's not on sys.path, then try to import and use it

file_to_import = sample_directory_name + '/' + sample_module_name + '.py'

try:
  os.mkdir( sample_directory_name )
  try:
    file_to_import_fd = open( file_to_import, CREATE_AS_NEW_FILE )
    try:
      return_message = f'{sample_module_name}.{sample_function_name} in {sample_directory_name}: x+y is {{x+y}}' 
      function_body  = "return f'" + return_message + "'" 
      function       = f'def {sample_function_name}(x, y):' + '\n  ' + function_body + '\n' 
      file_to_import_fd.write( function )
      file_to_import_fd.close()
      try:
        # update path before accessing the module 
        sys.path += [sample_directory_name]
        exec( f'import {sample_module_name}' )
        result_from_sample_function = eval( f'{sample_module_name}.{sample_function_name}(3, 4)' )
        print( result_from_sample_function )
      except Exception as exception:
        print( f"can't access {sample_module_name}: {make_printable(exception)}" )
      finally:
        # undo changes to the environment 
        sys.path.remove( sample_directory_name )
        del sys.modules[ sample_module_name ]
        importlib.invalidate_caches()
    except Exception as exception:
      print( f"can't write to file ({file_to_import}): {make_printable(exception)}" )
    finally:
      file_to_import_fd.close()  # in case a write failed
  except Exception as exception:
    print( f"can't create file ({file_to_import}): {make_printable(exception)}" )
  finally:
    try:
      shutil.rmtree( sample_directory_name )  # clean up the directory tree
    except Exception as exception:
      print( f"can't remove directory ({sample_directory_name}): {make_printable(exception)}" )
except Exception as exception:
  print( f"can't create directory ({sample_directory_name}): {make_printable(exception)}" )

**Exercises**:
-  Describe and account for the effect, if any, of removing the first call to `file_1_fd.close()` from this example.

##  11.2  Module caching and reimportation <a name='Modules-Caching-And-Reimportation'></a>
When Python first imports a module 'foo', it caches two of foo's attributes in `sys.modules`:
-  the name of the module being imported, in a key in `sys.modules`
-  the module's location, in the key's corresponding value

Subsequent attempts to reimport 'foo' then either
-  have no effect, if foo is present in the session, or
-  reference the module location recorded in sys.modules independently of `sys.path`, otherwise.

The import statement's caching of previously imported modules speeds the importation of new modules when those modules, 
in turn, reference previously imported modules.  Caching, however, can also hamper the updating of code over the course of a given session.

It is, however, possible to reset a module dynamically by
-  first using `importlib.invalidate_caches` to invalidate the python cache
-  then using `importlib.reload` to reload the module.

In [ ]:
# 11.2  Showing effect of import path and cache management on module reloading
#
# supporting constants
CREATE_AS_NEW_FILE = 'x'    # file access mode for open()

# library resources
import os, sys, importlib, shutil

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

# precondition check: ensure that we're creating a totally new, unreferenced module in a totally new directory

sample_directory_name_1 = 'sample_directory_1'
corrective_action = f'please change {sample_directory_name_1} to a directory name that is not in sys.path'
assert sample_directory_name_1 not in sys.path, corrective_action

sample_directory_name_2 = 'sample_directory_2'
corrective_action = f'please change {sample_directory_name_2} to a directory name that is not in sys.path'
assert sample_directory_name_2 not in sys.path, corrective_action

sample_module_name = 'sample_module' 
corrective_action = f'please change {sample_module_name} to a module name that is not in use'
assert sample_module_name not in sys.modules.keys(), corrective_action

sample_function_name = 'sample_function' 
corrective_action = f'please change {sample_function_name} to a function name that is not in use'
assert sample_function_name not in dir(), corrective_action

# preconditions established: create a module that's not on sys.path, then try to import it

file_to_import_1 = sample_directory_name_1 + '/' + sample_module_name + '.py'
file_to_import_2 = sample_directory_name_2 + '/' + sample_module_name + '.py'

try:
  os.mkdir( sample_directory_name_1 )
  try:
    file_to_import_1_fd = open( file_to_import_1, CREATE_AS_NEW_FILE )
    try:
      return_message = f'{sample_module_name}.{sample_function_name} in {sample_directory_name_1}: x+y is {{x+y}}' 
      function_body  = "return f'" + return_message + "'" 
      function       = f'def {sample_function_name}(x, y):' + '\n  ' + function_body + '\n' 
      file_to_import_1_fd.write( function )
      file_to_import_1_fd.close()
      try:
        os.mkdir( sample_directory_name_2 )
        try:
          file_to_import_2_fd = open( file_to_import_2, CREATE_AS_NEW_FILE ) 
          try:
            return_message = f'{sample_module_name}.{sample_function_name} in {sample_directory_name_2}: x-y is {{x-y}}' 
            function_body  = "return f'" + return_message + "'" 
            function       = f'def {sample_function_name}(x, y):' + '\n  ' + function_body + '\n' 
            file_to_import_2_fd.write( function )
            file_to_import_2_fd.close()
            try:
              # give precedence to sample_directory_name_1, then import and access the module 
              sys.path += [sample_directory_name_1, sample_directory_name_2] 
              print( '>> importing', sample_module_name, '-', sample_directory_name_1, 'precedes', sample_directory_name_2, 'in path' )
              exec( f'import {sample_module_name}' )
              result_from_sample_function = eval( f'{sample_module_name}.{sample_function_name}(3, 4)' )
              print( result_from_sample_function, '\n' )
              #
              # give precedence to sample_directory_name_2, then import and access the module 
              print( '>> reversing', sample_directory_name_1, 'and', sample_directory_name_2, 'in path' )
              sys.path = sys.path[:len(sys.path)-2] + [sample_directory_name_2, sample_directory_name_1] 
              result_from_sample_function = eval( f'{sample_module_name}.{sample_function_name}(3, 4)' )
              print( result_from_sample_function, '\n' )
              #
              # retry previous, but resetting and reloading Python's import cache first
              print( '>> invalidating cache and reimporting', sample_module_name )
              importlib.invalidate_caches() 
              exec( f'importlib.reload( {sample_module_name} )' ) 
              result_from_sample_function = eval( f'{sample_module_name}.{sample_function_name}(3, 4)' )
              print( result_from_sample_function )
            except Exception as exception:
              print( f"can't access {sample_module_name}: {make_printable(exception)}" )
            finally:
              # undo changes to the environment 
              sys.path.remove( sample_directory_name_1 )
              sys.path.remove( sample_directory_name_2 )
              del sys.modules[ sample_module_name ]
              importlib.invalidate_caches()
          except Exception as exception:
            print( f"can't write to file ({file_to_import_2}): {make_printable(exception)}" )
          finally:
            file_to_import_2_fd.close()
        except Exception as exception:
          print( f"can't create file ({file_to_import_2}): {make_printable(exception)}" )
        finally:
          try:
            shutil.rmtree( sample_directory_name_2 )
          except Exception as exception:
            print( f"can't remove directory ({sample_directory_name_2}): {make_printable(exception)}" )
      except Exception as exception:
        print( f"can't create directory ({sample_directory_name_2}): {make_printable(exception)}" )
    except Exception as exception:
      print( f"can't write to file ({file_to_import_1}): {make_printable(exception)}" )
    finally:
      file_to_import_1_fd.close()  # in case a write failed
  except Exception as exception:
    print( f"can't create file ({file_to_import_1}): {make_printable(exception)}" )
  finally:
    try:
      shutil.rmtree( sample_directory_name_1 )  # clean up the first directory tree
    except Exception as exception:
      print( f"can't remove directory ({sample_directory_name_1}): {make_printable(exception)}" )
except Exception as exception:
  print( f"can't create directory ({sample_directory_name_1}): {make_printable(exception)}" )